In [2]:
cd C:\Users\hannag01\Documents\wsm-calibration\

C:\Users\hannag01\Documents\wsm-calibration


c:\Users\hannag01\Documents\parallel-bayesopt-experiments\venv\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
import pandas as pd
import os
import numpy as np
from ax.service.ax_client import AxClient, ObjectiveProperties

In [33]:
from src.problems.surrogate import SurrogateProblem
from src.problems.surrogate_local import MultiFolderSurrogateProblem

In [34]:
# folder_path = r"C:\Users\hannag01\OneDrive - NHS Scotland\Whole System Modelling\Calibration\Highland\Highland-2024-07-10"
# folder_path = r"C:\Users\hannag01\OneDrive - NHS Scotland\Whole System Modelling\Calibration\Benchmarks\Highland-2024-07-16_1_a"
folder_path = r"C:\Users\hannag01\OneDrive - NHS Scotland\Whole System Modelling\Calibration\Benchmarks\Highland-2024-07-16_11_*"

In [35]:
# p = SurrogateProblem(folder_path)
p = MultiFolderSurrogateProblem(folder_path)

c:\Users\hannag01\Documents\parallel-bayesopt-experiments\venv\lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k2__length_scale is close to the specified upper bound 100000000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


In [36]:
p.gps['delay-output'].kernel.get_params()

{'k1': 1**2 * Matern(length_scale=[1, 1, 1], nu=2.5),
 'k2': WhiteKernel(noise_level=1),
 'k1__k1': 1**2,
 'k1__k2': Matern(length_scale=[1, 1, 1], nu=2.5),
 'k1__k1__constant_value': 1.0,
 'k1__k1__constant_value_bounds': (0.01, 100000000.0),
 'k1__k2__length_scale': array([1., 1., 1.]),
 'k1__k2__length_scale_bounds': [(0.01, 100000000.0),
  (0.01, 100000000.0),
  (0.01, 100000000.0)],
 'k1__k2__nu': 2.5,
 'k2__noise_level': 1.0,
 'k2__noise_level_bounds': (1e-10, 100.0)}

In [30]:
mygp = p.gps['delay-output']

In [37]:
p.gps['delay-output'].kernel_

1.27**2 * Matern(length_scale=[34.9, 26.2, 5.23e+04], nu=2.5) + WhiteKernel(noise_level=0.0175)

In [38]:
p.gps['emergency-output'].kernel_

4.08**2 * Matern(length_scale=[1e+08, 26.5, 2.09], nu=2.5) + WhiteKernel(noise_level=0.000301)

In [7]:
def prop_from_actual(value, actual):
    return np.absolute(value - actual) / actual

In [8]:
prop_from_actual(387.60852666, 381.0)

0.01734521433070865

In [9]:
np.absolute(387.60852666 - 381.0)

6.6085266599999954

In [10]:
p.config['actuals']

{'emergency': 381.0, 'delays': 104.0}

In [11]:
ax_client = AxClient()
ax_client.create_experiment(
    parameters=[
        {
            "name": "dd-los",
            "type": "range",
            "bounds": [20.0, 80.0],
            "value_type": "float",  
            "log_scale": False,
        },
        {
            "name": "gw-non-covid-los",
            "type": "range",
            "bounds": [5.0, 8.0],
        },
        {
            "name": "syswatch-scale",
            "type": "range",
            "bounds": [0.75, 1.0],
        }
    ],
    name="ax_surrogate_default_multi",
    objectives={"emergency_from_actual": ObjectiveProperties(minimize=True, threshold=0.05), "delays_from_actual": ObjectiveProperties(minimize=True, threshold=0.05)},
    tracking_metric_names=['emergencies', 'delays', 'error']
)

[INFO 07-23 12:11:20] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.
[INFO 07-23 12:11:20] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter gw-non-covid-los. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 07-23 12:11:20] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter syswatch-scale. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 07-23 12:11:20] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[RangeParameter(name='dd-los', parameter_type=FLOAT, range=[20.0, 80.0]), RangeParameter(name='gw-non-covid-los', parameter_type=FLOAT, range=[5.0, 8.0]), R

In [12]:
def mahan(values, means):
    '''Mahanalobis distance, based on 5% being 3 SDs away'''
    vars = [(1./3600)*(m**2) for m in means]
    sqer = [((values[i] - m)**2)/vars[i] for i, m in enumerate(means)]
    return sum(sqer)


In [13]:
for i in range(20):
    parameterization, trial_index = ax_client.get_next_trial()
    sample = p.sample_outputs([parameterization.get(ipt) for ipt in p.inputs])
    trial_result = {'emergency_from_actual': prop_from_actual(sample['emergency-output'][0], p.config['actuals']['emergency'])
                    , 'delays_from_actual': prop_from_actual(sample['delay-output'][0], p.config['actuals']['delays'])
                    , 'emergencies': sample['emergency-output'][0]
                    , 'delays': sample['delay-output'][0]
                    , 'error': mahan([sample['emergency-output'][0], sample['delay-output'][0]], [p.config['actuals']['emergency'], p.config['actuals']['delays']])
                    }
    ax_client.complete_trial(trial_index=trial_index, raw_data=trial_result)
    # ax_client.save_to_json_file(os.path.join(output_folder, 'ax_default', "ax_client_after_trial_{}.json".format(i)))

[INFO 07-23 12:11:21] ax.service.ax_client: Generated new trial 0 with parameters {'dd-los': 69.481016, 'gw-non-covid-los': 6.005222, 'syswatch-scale': 0.776319} using model Sobol.
[INFO 07-23 12:11:21] ax.service.ax_client: Completed trial 0 with data: {'emergency_from_actual': (0.182525, None), 'delays_from_actual': (0.085149, None), 'emergencies': (450.542069, None), 'delays': (95.144525, None), 'error': (146.03665, None)}.
[INFO 07-23 12:11:21] ax.service.ax_client: Generated new trial 1 with parameters {'dd-los': 30.040408, 'gw-non-covid-los': 7.142686, 'syswatch-scale': 0.776483} using model Sobol.
[INFO 07-23 12:11:21] ax.service.ax_client: Completed trial 1 with data: {'emergency_from_actual': (0.357352, None), 'delays_from_actual': (0.638582, None), 'emergencies': (517.15113, None), 'delays': (37.58745, None), 'error': (1927.755809, None)}.
[INFO 07-23 12:11:21] ax.service.ax_client: Generated new trial 2 with parameters {'dd-los': 62.54348, 'gw-non-covid-los': 6.682563, 'sysw

In [14]:
ax_client.get_pareto_optimal_parameters()

OrderedDict([(18,
              ({'dd-los': 69.43576291838708,
                'gw-non-covid-los': 5.0,
                'syswatch-scale': 0.88001172400287},
               ({'delays': 103.07744654698064,
                 'delays_from_actual': 0.020835363767784137,
                 'emergencies': 389.909316611623,
                 'emergency_from_actual': 0.023384036456086665,
                 'error': 3.4853167322953595},
                {'delays': {'delays': 5.582497281462046,
                  'delays_from_actual': 0.0,
                  'emergencies': 0.0,
                  'emergency_from_actual': 0.0,
                  'error': 0.0},
                 'delays_from_actual': {'delays': 0.0,
                  'delays_from_actual': 0.00019230262479701555,
                  'emergencies': 0.0,
                  'emergency_from_actual': 0.0,
                  'error': 0.0},
                 'emergencies': {'delays': 0.0,
                  'delays_from_actual': 0.0,
                  'eme

In [15]:
ax_client.experiment.optimization_config.objective

MultiObjective(objectives=[Objective(metric_name="emergency_from_actual", minimize=True), Objective(metric_name="delays_from_actual", minimize=True)])

In [16]:
from ax.plot.pareto_frontier import plot_pareto_frontier
from ax.plot.pareto_utils import compute_posterior_pareto_frontier
# Plotting imports and initialization
from ax.utils.notebook.plotting import init_notebook_plotting, render

In [17]:
objectives = ax_client.experiment.optimization_config.objective.objectives
frontier = compute_posterior_pareto_frontier(
    experiment=ax_client.experiment,
    data=ax_client.experiment.fetch_data(),
    primary_objective=objectives[0].metric,
    secondary_objective=objectives[1].metric,
    absolute_metrics=["emergency_from_actual", "delays_from_actual"],
    num_points=20,
)

render(plot_pareto_frontier(frontier, CI_level=0.95))

In [18]:
ax_client.get_hypervolume()

0.0008080698854580234

In [19]:
from ax.plot.slice import plot_slice

In [20]:
model = ax_client.generation_strategy.model

In [21]:
render(plot_slice(model, "dd-los", "error"))

In [24]:
p.results_df[p.results_df['row-type'] == 'actual']

,Unnamed: 0,dd-los,gw-non-covid-los,syswatch-scale,emergency-output,delay-output,overall-error,output-csv-filename,emergency-within-5-perc,delay-within-5-perc,row-type
0,0,53.664804,6.897759,0.963086,509.285714,73.285714,722.130595,none,False,False,actual
1,1,31.036181,6.089260,0.876577,456.142857,43.571429,1355.435699,none,False,False,actual
2,2,36.699649,5.062375,0.887085,390.428571,50.428571,957.422190,none,True,False,actual
3,3,66.853683,5.441196,0.952527,418.857143,94.428571,66.034761,none,False,False,actual
4,4,51.462551,5.954117,0.964971,444.714286,68.428571,521.827636,none,False,False,actual
5,5,56.492846,7.342448,0.792696,528.000000,79.857143,729.909374,none,False,False,actual
6,6,23.479124,7.216818,0.996373,524.714286,28.142857,2427.479775,none,False,False,actual
7,7,42.523915,5.581884,0.848784,429.428571,58.714286,740.751613,none,False,False,actual
8,8,27.588600,6.841708,0.791770,501.142857,34.857143,1949.192375,none,False,False,actual
9,9,43.002638,6.936464,0.880822,505.142857,58.285714,1077.772867,none,False,False,actual


In [23]:
# To Do:
# Use surrogate model to get original data in the format needed
# Copy AX implementation of model with no data
# 

In [29]:
ax_client.generation_strategy.current_node.

GenerationStep(model=<Models.BOTORCH_MODULAR: 'BoTorch'>, num_trials=-1, model_kwargs={'torch_device': None, 'fit_out_of_design': False, 'transform_configs': {'Derelativize': {'use_raw_status_quo': False}, 'Winsorize': {'derelativize_with_raw_status_quo': False}}, 'transforms': [<class 'ax.modelbridge.transforms.winsorize.Winsorize'>, <class 'ax.modelbridge.transforms.remove_fixed.RemoveFixed'>, <class 'ax.modelbridge.transforms.choice_encode.OrderedChoiceToIntegerRange'>, <class 'ax.modelbridge.transforms.one_hot.OneHot'>, <class 'ax.modelbridge.transforms.int_to_float.IntToFloat'>, <class 'ax.modelbridge.transforms.log.Log'>, <class 'ax.modelbridge.transforms.logit.Logit'>, <class 'ax.modelbridge.transforms.unit_x.UnitX'>, <class 'ax.modelbridge.transforms.ivw.IVW'>, <class 'ax.modelbridge.transforms.derelativize.Derelativize'>, <class 'ax.modelbridge.transforms.standardize_y.StandardizeY'>]}, model_gen_kwargs=None, completion_criteria=[], min_trials_observed=0, max_parallelism=3, us

In [40]:
ax_client.generation_strategy.current_step.model_spec

ModelSpec(	model_enum=BoTorch,
	model_kwargs={"fit_out_of_design": false, "torch_device": null, "transform_configs": {"Derelativize": {"use_raw_status_quo": false}, "Winsorize": {"derelativize_with_raw_status_quo": false}}, "transforms": ["<class 'ax.modelbridge.transforms.winsorize.Winsorize'>", "<class 'ax.modelbridge.transforms.remove_fixed.RemoveFixed'>", "<class 'ax.modelbridge.transforms.choice_encode.OrderedChoiceToIntegerRange'>", "<class 'ax.modelbridge.transforms.one_hot.OneHot'>", "<class 'ax.modelbridge.transforms.int_to_float.IntToFloat'>", "<class 'ax.modelbridge.transforms.log.Log'>", "<class 'ax.modelbridge.transforms.logit.Logit'>", "<class 'ax.modelbridge.transforms.unit_x.UnitX'>", "<class 'ax.modelbridge.transforms.ivw.IVW'>", "<class 'ax.modelbridge.transforms.derelativize.Derelativize'>", "<class 'ax.modelbridge.transforms.standardize_y.StandardizeY'>"]},
	model_gen_kwargs={},
	model_cv_kwargs={},
)

In [41]:
new_model = ax_client.generation_strategy.current_step

In [53]:
m = new_model.model(search_space=ax_client.experiment.search_space, data=ax_client.experiment.fetch_data())

DataRequiredError: `StandardizeY` transform requires non-empty data.

In [54]:
same_data = ax_client.experiment.fetch_data()

In [55]:
same_data

In [56]:
new_experiment = ax_client.experiment.clone_with(data=same_data)

In [60]:
ax_client2 = AxClient()

[INFO 07-23 12:35:03] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.


In [63]:
ax_client2.

TypeError: 'Experiment' object is not iterable

In [46]:
new_model.model._generate_next_value_()

TypeError: Enum._generate_next_value_() missing 3 required positional arguments: 'start', 'count', and 'last_values'

In [52]:
ax_client.experiment.fetch_data().df

,arm_name,metric_name,mean,sem,trial_index
0,0_0,emergency_from_actual,0.182525,NaN,0
1,0_0,delays_from_actual,0.085149,NaN,0
2,0_0,emergencies,450.542069,NaN,0
3,0_0,delays,95.144525,NaN,0
4,0_0,error,146.036650,NaN,0
...,...,...,...,...,...
95,19_0,emergency_from_actual,0.024440,NaN,19
96,19_0,delays_from_actual,0.042528,NaN,19
97,19_0,emergencies,390.311731,NaN,19
98,19_0,delays,99.577139,NaN,19


In [68]:
# from ax.modelbridge.factory import get_MOO_NEHVI
from ax.modelbridge.registry import Models
from ax.modelbridge.modelbridge_utils import observed_hypervolume

In [75]:
ax_client.get_pareto_optimal_parameters()

OrderedDict([(18,
              ({'dd-los': 69.43576291838708,
                'gw-non-covid-los': 5.0,
                'syswatch-scale': 0.88001172400287},
               ({'delays': 103.07744654698064,
                 'delays_from_actual': 0.020835363767784137,
                 'emergencies': 389.909316611623,
                 'emergency_from_actual': 0.023384036456086665,
                 'error': 3.4853167322953595},
                {'delays': {'delays': 5.582497281462046,
                  'delays_from_actual': 0.0,
                  'emergencies': 0.0,
                  'emergency_from_actual': 0.0,
                  'error': 0.0},
                 'delays_from_actual': {'delays': 0.0,
                  'delays_from_actual': 0.00019230262479701555,
                  'emergencies': 0.0,
                  'emergency_from_actual': 0.0,
                  'error': 0.0},
                 'emergencies': {'delays': 0.0,
                  'delays_from_actual': 0.0,
                  'eme

In [66]:
ehvi_model = Models.BOTORCH_MODULAR(
    experiment=ax_client.experiment,
    data=same_data,
)

In [67]:
ehvi_model

TorchModelBridge(model=BoTorchModel)

In [69]:
hv = observed_hypervolume(modelbridge=ehvi_model)
hv

0.0012286908178038224

In [70]:
ax_client.get_hypervolume()

0.0008080698854580234

In [71]:
ax_client.get_hypervolume()

0.0008080698854580234

In [76]:
same_data.df

,arm_name,metric_name,mean,sem,trial_index
0,0_0,emergency_from_actual,0.182525,NaN,0
1,0_0,delays_from_actual,0.085149,NaN,0
2,0_0,emergencies,450.542069,NaN,0
3,0_0,delays,95.144525,NaN,0
4,0_0,error,146.036650,NaN,0
...,...,...,...,...,...
95,19_0,emergency_from_actual,0.024440,NaN,19
96,19_0,delays_from_actual,0.042528,NaN,19
97,19_0,emergencies,390.311731,NaN,19
98,19_0,delays,99.577139,NaN,19


In [80]:
same_data._get_records()

TypeError: Data._get_records() missing 2 required positional arguments: 'evaluations' and 'trial_index'

In [81]:
ax_client.attach_trial

Signature:
ax_client.parameter_from_json(
    representation: Dict[str, Union[NoneType, str, bool, float, int, Sequence[Union[NoneType, str, bool, float, int]], Dict[str, List[str]]]],
) -> ax.core.parameter.Parameter
Docstring: Instantiate a parameter from JSON representation.
File:      c:\users\hannag01\documents\parallel-bayesopt-experiments\venv\lib\site-packages\ax\service\utils\instantiation.py
Type:      method

In [82]:
same_data.df

,arm_name,metric_name,mean,sem,trial_index
0,0_0,emergency_from_actual,0.182525,NaN,0
1,0_0,delays_from_actual,0.085149,NaN,0
2,0_0,emergencies,450.542069,NaN,0
3,0_0,delays,95.144525,NaN,0
4,0_0,error,146.036650,NaN,0
...,...,...,...,...,...
95,19_0,emergency_from_actual,0.024440,NaN,19
96,19_0,delays_from_actual,0.042528,NaN,19
97,19_0,emergencies,390.311731,NaN,19
98,19_0,delays,99.577139,NaN,19


In [39]:
ax_client

NameError: name 'ax_client' is not defined